# What are the details of one of my _apps_?
### Overview
Here we focus on _getting the details_ from a single app. This will be important later when we want to start a task. As with any **detail**-type call, we will get extensive information about one app but must first know that app's id. We will show two options (A & B) of getting the same information.

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="set_AUTH_TOKEN.ipynb">**set_AUTH_TOKEN.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly here).
 4. You understand how to <a href="apps_listAll.ipynb" target="_blank">list</a> apps within one of your projects (we will just use that call directly here).
 5. You have added (or  <a href="apps_copyFromPublicApps.ipynb" target="_blank">copied</a>) least one app file in your project
 
### Imports and Definitions
 A single call is sufficient. We will show two different options, both of which are defined in the apimethods.py file.

In [ ]:
from defs.apimethods import *

## (Option A) Use API() _object_
We start by listing all of your projects, then list the apps within the first one. A **detail**-call for projects returns the following *attributes*:
* **id**     _Unique_ identifier for the app
* **name**   Name of the app, note this **is** metadata and can be _changed_
* **href**   Address<sup>1</sup> of the app.
* **raw**    Details of the app
* **project** Project the app is in
* **revision** App revision
* **flag**   (unimportant) this is set by the object constructor, here always 'longList':False 

The **raw** dictionary<sup>2</sup>:
* stdout
* sbg:categories - categories that app belongs to 
* sbg:copyOf - what is the app this one was copied from
* sbg:toolAuthor - author of the tool
* id - app id
* successCodes - list of success codes for a tool
* sbg:revision - revision of the app
* sbg:latestRevision - latest revision available
* sbg:modifiedBy - user who modified the app revision 
* sbg:toolkitVersion - version of the toolkit 
* label - User friendly naming of the app (like *Picard MergeSamFiles*)
* arguments
* sbg:createdBy - which user originally created the app
* inputs - description of the inputs 
* sbg:links - links to original documentation for the tools
* description 
* sbg:sbgMaintained - notes whether the app is maintained by Seven Bridges or not
* sbg:modifiedOn - modified info timestamp
* outputs - description of the outputs
* baseCommand - base command that executes (for example `java -jar /opt/picard-tools-1.140/picard.jar MergeSamFiles`, but in more complex format)
* sbg:contributors - list of contributors to the app
* sbg:validationErrors - app validation errors if they exist
* requirements 
* temporaryFailCodes - codes that cause temporary failure (which can be retried)
* class 
* sbg:job 
* hints - contains info on memory, CPU requirements and the link to actual *docker* image that executes
* sbg:id 
* sbg:createdOn - timestamp when the app was created
* sbg:toolkit - toolkit version 
* stdin 
* sbg:cmdPreview - preview of the command line - like `java -Xmx2048M -jar /opt/picard-tools-1.140/picard.jar CollectAlignmentSummaryMetrics INPUT=/root/folder/example.bam REFERENCE_SEQUENCE=/root/directory/example.fasta OUTPUT=example.summary_metrics.txt METRIC_ACCUMULATION_LEVEL=ALL_READS INPUT=/root/folder/example.bam /root/folder/example.bam`
* sbg:license - licencing for the tool 
* sbg:revisionsInfo - contains info on revisions
* sbg:project - what is the project that contains the app

<sup>1</sup> This is the address where, by using API you can get this resource

<sup>2</sup> Many of these fields are related to the [Common Workflow Language](http://docs.cancergenomicscloud.org/docs/sdk-overview) JSON describing the app.

In [ ]:
# [USER INPUT] Set project (p_) and app (a_) indices:
p_index = 2                               # project to check
a_index = 0                               # app index

# LIST all projects
existing_projects = API('projects')                              

# list the apps in ONE project
my_apps = API(path='apps', query={'project': existing_projects.id[p_index]})

# get details of the app
single_app = API(path=('apps/' + my_apps.id[a_index]))

print('You have selected app (%s), which was created by (%s). \n It has %i inputs:' % \
      (single_app.name, single_app.raw['sbg:toolAuthor'], len(single_app.raw['inputs'])))
for ii in single_app.raw['inputs']:
    print(ii['id'][1:])

## Get a dictionary of the App's ~CWL
This is a useful **first look** at the app description, but please **be careful** since we have wrapped it in a function, it is **not a JSON**. You would want to 

1. go to the GUI
2. click on the app
3. in the top right, click "Export"
4. **save** that page as a json. (do not copy paste as that introduces surpsring errors)
5. load the json file into python.
6. convert it into a json (see this [tutorial](../../Tutorials/CGC/batch_SAMtoolsView.ipynb) for a good example)

In [ ]:
# get (and print) raw CWL of the app
api_call(path=('apps/' + my_apps.id[a_index] + '/raw'))

## (Option B) Use api_call() *function*
Using the api_call() function returns the same information as above, but now as *keys* within a dictionary for each project. To start, we list the first 25 projects you are a member of. Then, we query the _file list_ for the first project.

<sup>2</sup> Note that each entry in the 'items' list has the complete information for a single member, unlike the object above which organizes and concantenates all the dictionaries.

In [ ]:
# [USER INPUT] Set project (p_) and app (a_) indices:
p_index = 2                               # project to check
a_index = 0                               # app index

# Short (n=26) list of projects I am a member of
existing_projects = api_call(path='projects', method='GET', \
                            query={'limit':26, 'offset':0}, data=None)      

# apps within my first project
my_apps = api_call(path='apps', \
                   query={'limit':25, \
                          'project': existing_projects['items'][p_index]['id']})

# get details of the first app inside that project
api_call(path=('apps/' + my_apps['items'][a_index]['id']))

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.cancergenomicscloud.org/docs/get-file-details) and [here](http://docs.cancergenomicscloud.org/docs/get-raw-app-information)